In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2 
from PIL import Image
import kagglehub
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

c:\Users\naqsh\.conda\envs\dl_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:

# Download latest version
path = kagglehub.dataset_download("meowmeowmeowmeowmeow/gtsrb-german-traffic-sign")


print("Path to dataset files:", path)

100%|██████████| 612M/612M [00:59<00:00, 10.8MB/s] 

Extracting files...


Path to dataset files: C:\Users\naqsh\.cache\kagglehub\datasets\meowmeowmeowmeowmeow\gtsrb-german-traffic-sign\versions\1


In [25]:
data = ImageDataGenerator(rescale=1/255, validation_split=0.2)
train= data.flow_from_directory(
    path+ "/Train",
    target_size = (64,64),
    batch_size= 16,
    class_mode = "categorical",
    subset = "training"
)
val= data.flow_from_directory(
    path+ "/Train",
    target_size = (64,64),
    batch_size= 16,
    class_mode = "categorical",
    subset = "validation",
    shuffle = False,
)

Found 31368 images belonging to 43 classes.
Found 7841 images belonging to 43 classes.


In [27]:
model =models.Sequential([
    layers.Input(shape=(64,64,3)),
    
    layers.Conv2D(32,(3,3), padding = 'same'),
    layers.Activation('elu'),
    layers.MaxPooling2D(2,2),
    Dropout(0.2),
    
    layers.Conv2D(64,(3,3), padding = 'same'),
    layers.Activation('elu'),
    layers.MaxPooling2D(2,2),
    Dropout(0.2),


    layers.Conv2D(128,(3,3), padding = 'same'),
    layers.Activation('elu'),
    layers.MaxPooling2D(2,2),
    Dropout(0.2),

    
    layers.Flatten(),
    layers.Dense(256),
    layers.Activation('elu'),
    layers.Dense(43, activation='softmax')  
])
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)              │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_13 (Activation)      │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_14 (Activation)      │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_15 (Activation)      │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │     2,097,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_16 (Activation)      │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 43)             │        11,051 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,201,707 (8.40 MB)

 Trainable params: 2,201,707 (8.40 MB)

 Non-trainable params: 0 (0.00 B)

In [29]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [30]:

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5,          
    patience=3,          
    verbose=1,
    min_lr=1e-6
)

In [ ]:
trained_model = model.fit(
    train,
    validation_data = val,
    epochs = 20,
    callbacks = [early_stop,lr_scheduler]
)

Epoch 1/20
1961/1961 ━━━━━━━━━━━━━━━━━━━━ 217s 108ms/step - accuracy: 0.8598 - loss: 0.5111 - val_accuracy: 0.8630 - val_loss: 0.6402 - learning_rate: 0.0010
Epoch 2/20
1961/1961 ━━━━━━━━━━━━━━━━━━━━ 210s 107ms/step - accuracy: 0.9620 - loss: 0.1646 - val_accuracy: 0.8989 - val_loss: 0.6681 - learning_rate: 0.0010
Epoch 3/20
 261/1961 ━━━━━━━━━━━━━━━━━━━━ 3:17 116ms/step - accuracy: 0.9649 - loss: 0.1342

In [ ]:
val_preds = model.predict(val)
val_pred_classes = np.argmax(val_preds, axis=1)

true_classes = val.classes
class_labels = list(val.class_indices.keys())


print(classification_report(true_classes, val_pred_classes, target_names=class_labels))

491/491 ━━━━━━━━━━━━━━━━━━━━ 14s 28ms/step
              precision    recall  f1-score   support

           0       0.92      0.83      0.88        42
           1       0.85      0.82      0.83       444
          10       0.98      0.99      0.98       402
          11       0.92      0.95      0.94       264
          12       0.98      0.99      0.99       420
          13       1.00      0.97      0.98       432
          14       0.76      1.00      0.87       156
          15       0.98      0.74      0.84       126
          16       1.00      0.98      0.99        84
          17       0.99      0.98      0.99       222
          18       0.80      0.91      0.85       240
          19       0.88      0.90      0.89        42
           2       0.97      0.83      0.89       450
          20       0.97      0.54      0.70        72
          21       0.96      0.98      0.97        66
          22       0.95      0.99      0.97        78
          23       0.78      0.95     

In [ ]:
model.save("")